In [1]:
import sys
sys.path += ['../..']
sys.path += ['..']

from data_collection.data_collection import LoggerSet, Logger

import numpy as np
import pandas as pd
import plotly.express as px
from data_collection.video_data import get_frame_iterator
from pathlib import Path
from typing import Iterable, Tuple, List
from tqdm import tqdm
import datetime
import tensorflow as tf
import tensorflow.keras as keras

%load_ext autoreload
%autoreload 2


2024-07-10 20:06:43.439886: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-10 20:06:44.072070: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Note


# Loading data 

In [2]:
from with_linear_acc_v2 import Session
sessions_v0 = Session.load_multiple_session('../data/v0',         
    camera_log_name = 'PicameraV2', 
    angular_speed_control_log_name='AngularSpeedControlV2'
)
sessions_v30Jun = Session.load_multiple_session('../data/v30Jun')


[mov,mp4,m4a,3gp,3g2,mj2 @ 0x62e2dcce1dc0] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x62e2ddf1c200] moov atom not found
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x62e2e25feb40] moov atom not found


In [3]:
def match_time(t1_col, t2_col, t1_offset_ms=0, t2_offset_ms=0, sort_check=True):
    """
    return the indices of t1_col that match t2
    """
    if sort_check: 
        sortedt1 = np.sort(t1_col)
        sortedt2 = np.sort(t2_col)
        sortedt1 = sortedt1[~np.isnan(sortedt1)]
        sortedt2 = sortedt2[~np.isnan(sortedt2)]

        assert np.all(t1_col[~np.isnan(t1_col)] == sortedt1)
        assert np.all(t2_col[~np.isnan(t2_col)] == sortedt2)
    return np.searchsorted(t1_col+pd.to_timedelta(t1_offset_ms, unit='ms'), t2_col+pd.to_timedelta(t2_offset_ms, unit='ms'))

In [4]:
def rotate_vector(v1, theta):
    """
    auther: chatgpt
    """
    v1 = np.array(v1)
    x1, y1 = v1[...,0], v1[...,1]
    
    x2 = x1 * np.cos(theta) - y1 * np.sin(theta)
    y2 = x1 * np.sin(theta) + y1 * np.cos(theta)
    
    return np.stack([x2, y2], axis=-1)

In [5]:
def mean_nominmax(x):
    return (x.sum()-x.max()-x.min())/(len(x)-2)

def get_rotation_angle(v2, v1):
    """
    v1 is the reference angle 
    auther: chatgpt
    """
    v1 = np.array(v1)
    v2 = np.array(v2)
    x1, y1 = v1[...,0], v1[...,1]
    x2, y2 = v2[...,0], v2[...,1]
    
    dot_product = x1 * x2 + y1 * y2
    cross_product = y2 * x1 - x2 * y1
    theta = np.arctan2(cross_product, dot_product)
    return theta

def data_prep(obj: Session):

    adf = obj.data['angular_speed_control_df']
    cdf = obj.data['camera_df']

    def add_angle_offset_columns():

                
        offset_list = [0, 200, 400, 800]
        tidxs = [
            match_time(adf['time_AngularSpeedControl'], cdf['time'],  t2_offset_ms=_offset)
            for _offset in offset_list
        ]
        col_names: List = ['angle']
        omega_shifted = np.concatenate([
                adf[col_names].values[_tidx] 
                for _tidx in tidxs
            ], axis=1)

        column_names = [c+'_'+str(t) for t in offset_list for c in col_names]
        cdf[column_names] = omega_shifted
        obj.data['add_angle_offset_columns'] = column_names
        obj.data['offset_list'] = offset_list

        return offset_list, column_names


    def add_sin_cos_diff(offset_list, angle_col_names):
        as_rad = cdf[angle_col_names].values/180*np.pi

        #sin_col = [c+'_sin' for c in angle_col_names]
        #cos_col = [c+'_cos' for c in angle_col_names]

        sine = np.sin(as_rad)
        cosine = np.cos(as_rad)
        ref_angle = np.stack([cosine[..., 0], sine[..., 0]], axis=-1)

        rotations = []
        for i in range(1, len(angle_col_names)):
            angle = np.stack([cosine[..., i], sine[..., i]], axis=-1)
            rotations.append(
                get_rotation_angle(angle, ref_angle)
            )
        
        rotations = np.stack(rotations, axis=-1)

        col_names = ['rotation_'+str(o) for o in offset_list[1:]]
        
        cdf[col_names] = rotations
        
        obj.data['add_sin_cos_diff'] =   col_names

    offset_list, angle_col_names = add_angle_offset_columns()
    add_sin_cos_diff(offset_list, angle_col_names)

    def add_speed_column():

        tidx = match_time(cdf['time'],  adf['time_AngularSpeedControl'])
        adf['frame_match'] = tidx
        mean_speed = adf.groupby('frame_match')['speed'].mean()

        # this is the mean speed after the end (acquisition) of the frame until the end of next frame
        cdf['mean_speed'] = mean_speed

    add_speed_column()

    

def sample_prep(obj: Session):
    frames, camera_df = obj.data['frames'], obj.data['camera_df']

    camera_df = camera_df.query('mean_speed>0')
    out = camera_df[obj.data['add_sin_cos_diff']]
    
    frames = frames[out.index]

    obj.samples = frames, out.values/np.pi
    


def stack_samples(objs: List[Session], sample_axis=0):
    samples_stacked = []
    for s in zip(*[o.samples for o in objs]):
        samples_stacked.append(np.concatenate(s, axis=sample_axis))
    return samples_stacked



In [6]:
[data_prep(s) for s in tqdm(sessions_v30Jun+sessions_v0)]; 
[sample_prep(s) for s in tqdm(sessions_v30Jun+sessions_v0)]; 

x, y = stack_samples(sessions_v30Jun+sessions_v0)

from train_tools import validation_chunk_split
train_idx, val_idx = validation_chunk_split(len(x), val_split=0.2)
train_x, train_y = x[train_idx], y[train_idx]
val_x, val_y = x[val_idx], y[val_idx]


100%|██████████| 4/4 [00:00<00:00, 57.54it/s]


## Inspection only

In [ ]:
rotate_vector

In [ ]:
sessions_v30Jun[0].data['camera_df']['rotation_800']

In [ ]:
px.line(sessions_v30Jun[1].samples[1])

In [ ]:
df_vis = sessions[0].data['angular_speed_control_df']

In [ ]:

def mean_nominmax(x):
    return (x.sum()-x.max()-x.min())/(len(x)-2)
asd0 = df_vis['angular_velocity'].rolling(100).apply(mean_nominmax)#.shift(-99)

asd = df_vis['angular_velocity'].rolling(50).apply(mean_nominmax).shift(-25)
asd1 = df_vis['angular_velocity'].rolling(3).apply(mean_nominmax).shift(-2)


px.line(y=[asd1, asd, asd0])

In [ ]:
px.line(df_vis, y=['angular_velocity_smooth', 'angular_velocity'])

In [ ]:
px.imshow(sessions[0].data['frames'][-1000])

# Model

In [7]:
def get_model(lr=.1, other_metrics=[]):
    tf.keras.backend.clear_session()
    image_shape = 64, 114, 3

    img_aug_preprocess_layers = [
        keras.layers.RandomTranslation(0.05, 0.05, fill_mode='reflect'),
        keras.layers.RandomRotation(0.02, fill_mode='reflect'),
        keras.layers.RandomZoom(0.05, fill_mode='reflect'),
        keras.layers.RandomContrast(0.3),
        keras.layers.RandomBrightness(factor=0.6, value_range=[0, 1])
    ]
    
    layers =  [
        keras.layers.Conv2D(16, 5, strides=2, activation='relu'), 
        keras.layers.Conv2D(32, 5, strides=2, activation='relu'), 
        keras.layers.Conv2D(64, 5, strides=2, activation='relu'),         
        keras.layers.Conv2D(64, (5, 3), activation='relu'), 
        keras.layers.Flatten(),
        keras.layers.Dropout(0.2),
        
        keras.layers.Dense(64, activation='relu'), 
        keras.layers.Dense(64, activation='relu'), 
        keras.layers.Dense(3, activation='tanh'), 
    ]
    
    model = keras.Sequential(
        [
            keras.layers.InputLayer(image_shape),
            keras.layers.Rescaling(1/255), 
            *img_aug_preprocess_layers,
            *layers, 
        ])


    optimiser = keras.optimizers.Adam(lr)
    model.compile(optimizer=optimiser, loss='Huber', metrics=['MAE']+other_metrics)

    return model 

model = get_model()
model.summary()

2024-07-10 20:06:51.221184: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-10 20:06:51.254152: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-10 20:06:51.254401: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 64, 114, 3)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_translation              │ (None, 64, 114, 3)     │             0 │
│ (RandomTranslation)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation                 │ (None, 64, 114, 3)     │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom (RandomZoom)        │ (None, 64, 114, 3)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_contrast                 │ (None, 64, 114, 3)     │             0 │
│ (RandomContrast)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_brightness               │ (None, 64, 114, 3)     │             0 │
│ (RandomBrightness)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 30, 55, 16)     │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 26, 32)     │        12,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 5, 11, 64)      │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 1, 9, 64)       │        61,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 168,099 (656.64 KB)

 Trainable params: 168,099 (656.64 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
from train_tools import find_lr, tg_notify, use_tensorboard, end_epoch_notify

In [ ]:
find_lr(get_model(), train_x, train_y)
#tensorboard --logdir ./logs --bind_all

In [9]:
model = get_model(1e-4)
model.fit(
    train_x,
    train_y,
    epochs=1000, 
    validation_data = (val_x, val_y),
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),
        use_tensorboard('training'), 
        end_epoch_notify()
        ] 
    )

Epoch 1/1000


2024-07-10 20:06:57.872536: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


503/503 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - MAE: 0.0873 - loss: 0.0095 - val_MAE: 0.0823 - val_loss: 0.0087
Epoch 2/1000
503/503 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - MAE: 0.0838 - loss: 0.0089 - val_MAE: 0.0796 - val_loss: 0.0083
Epoch 3/1000
503/503 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - MAE: 0.0822 - loss: 0.0085 - val_MAE: 0.0688 - val_loss: 0.0066
Epoch 4/1000
503/503 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - MAE: 0.0764 - loss: 0.0075 - val_MAE: 0.0675 - val_loss: 0.0062
Epoch 5/1000
503/503 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - MAE: 0.0737 - loss: 0.0070 - val_MAE: 0.0622 - val_loss: 0.0055
Epoch 6/1000
503/503 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - MAE: 0.0713 - loss: 0.0067 - val_MAE: 0.0597 - val_loss: 0.0050
Epoch 7/1000
503/503 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - MAE: 0.0683 - loss: 0.0061 - val_MAE: 0.0618 - val_loss: 0.0049
Epoch 8/1000
503/503 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - MAE: 0.0677 - loss: 0.0059 - val_MAE: 0.0585 - val_loss: 0.0043
Epoch 9/1000
503/503 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step

In [ ]:
#frames_reindexed#, other_inputs_reindexed, outputs

# visualise the performance

In [10]:

#train_frames, train_other, train_out
val_idxv = np.sort(val_idx)
train_idxv = np.sort(train_idx)

val_y_pred = model.predict(x[val_idxv], batch_size=64)
train_y_pred = model.predict(x[train_idxv], batch_size=64)


val_outv = y[val_idxv]
train_outv = y[train_idxv]



px.line(np.c_[train_outv[:, 2], train_y_pred[:, 2]]*np.pi, title='train')

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
252/252 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [11]:
px.line(np.c_[val_outv[:,  2], val_y_pred[:,  2]]*np.pi, title='val_train')

In [12]:
#model.export('29Jun-export.keras')
model.save('model3-10Jul.keras')


In [ ]:
assert False, "sessions_v0 is now included in the training data"

x_val2, y_val2 = stack_samples(sessions_v0)

y_val2_pred = model.predict(x_val2, batch_size=64)


In [ ]:
px.line(np.c_[y_val2[:, 1], y_val2_pred[:, 1]]*np.pi, title='val2')